In [81]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-nov-2021/train.csv
/kaggle/input/tabular-playground-series-nov-2021/test.csv


In [ ]:
import os
import wandb
from wandb.keras import WandbCallback
os.system('! wandb login be213aaff4ff14945d480abc18697d8664bba8c8')
training = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/test.csv')
test['target'] = np.NaN
training['train_test'] = 1
test['train_test'] = 0
all_data = pd.concat([training,test]) 

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
attribute_names = all_data.columns.tolist()
test_atts = test.columns.tolist()
all_dummies = pd.get_dummies(all_data[attribute_names])
test_dummies = pd.get_dummies(test[test_atts])  
X_train = all_dummies[all_data.train_test == 1].drop(['train_test'], axis=1)
for column in X_train.columns:
    mean_value = X_train[column].mean()
    X_train[column].fillna(mean_value, inplace=True)
X_check = X_train #for heatmap
X_train = X_train.drop(['target'], axis=1)
X_backup = X_train
X_test = all_dummies[all_data.train_test == 0].drop(['train_test'], axis=1)
y_train = all_data[all_data.train_test==1].target
y_backup = y_train
X_test = X_test.drop(['target'], axis=1)
for column in X_test.columns:
    mean_value = X_test[column].mean()
    X_test[column].fillna(mean_value, inplace=True)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
training

In [ ]:
all_data

In [ ]:
X_train

In [ ]:
import matplotlib.pyplot as plt

# Load your data into a DataFrame, assuming your target variable is named 'target'
# Replace 'your_data.csv' with your actual data file path or source

# Create a countplot to visualize the distribution of the target variable
sns.set(style="whitegrid")  # Set the style of the plot

plt.figure(figsize=(8, 6))  # Set the figure size (width, height)
sns.countplot(data=training, x='target')  # Replace 'target' with your actual column name

# Customize the plot
plt.title('Distribution of Target Variable (0 and 1)')
plt.xlabel('Target Value')
plt.ylabel('Count')

# Show the plot
plt.show()
#even though it is a task about finding spam emails, the data is balanced

In [ ]:
X_test

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
X_adapt = X_train.drop(['id'], axis=1)
all_attributes =X_adapt.columns.tolist()
columns_to_normalize = all_attributes
scaler = MinMaxScaler()
X_train_scaled = X_train
X_train_scaled[columns_to_normalize] = scaler.fit_transform(X_train[columns_to_normalize])
#X_train_scaled = X_train_scaled[:,~np.all(np.isnan(d), axis=0)]
X_test_scaled = X_test
X_test_scaled[columns_to_normalize] = scaler.fit_transform(X_test[columns_to_normalize])

#also norm. backup
X_backup[columns_to_normalize] = scaler.fit_transform(X_backup[columns_to_normalize])

In [ ]:
X_train_scaled

In [ ]:
target_column = 'target'

# Calculate the correlations between the target and all features
correlations = training.corr()[target_column]
correlations = correlations.drop('target')
# Filter features with a correlation of at least 0.4 or -0.4
significant_features = correlations[(correlations >= 0.1) | (correlations <= -0.1)]
significant_features_index = correlations[(correlations >= 0.1) | (correlations <= -0.1)].index

# Visualize the correlation values
plt.figure(figsize=(10, 6))
sns.barplot(x=significant_features.values, y=significant_features.index)
plt.title(f'Features Correlating at least 0.1 or -0.1 with {target_column}')
plt.xlabel('Correlation')
plt.ylabel('Features')
plt.show()

In [ ]:
print(significant_features_index)

In [ ]:
X_train_sig = X_train_scaled[significant_features_index]
print(X_train_sig)

In [ ]:
print(X_backup)

In [ ]:
sample_fraction = 0.1

num_samples = int(len(X_train_sig) * sample_fraction)
num_samples_2 = int(len(training) * sample_fraction)

X_train_sig.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

random_indices = np.random.choice(len(X_train_sig), num_samples, replace=False)
random_indices_2 = np.random.choice(len(training), num_samples_2, replace=False)

X_subset = X_train_sig.iloc[random_indices]
y_subset = y_train.iloc[random_indices]

X_backup = X_backup.iloc[random_indices_2]
y_backup = y_backup.iloc[random_indices_2]

'''X_val = X_train.iloc[random_indices]
y_val = y_train.iloc[random_indices]'''

In [ ]:
print(X_subset)
print(y_subset)

In [ ]:
print(X_backup)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

'''wandb.init(project='KaggleVovember2021', name='untuned attemps')

def run_cross_validation(models, X_train, y_train, cv=5):
    results = []

    for model in models:
        model_name = model.__class__.__name__

        if model_name == 'XGBClassifier':
            class_counts = y_train.value_counts()
            class_weight = class_counts[0] / class_counts[1]
            model = XGBClassifier(scale_pos_weight=class_weight, random_state=1)
        
        cv_scores = cross_val_score(model, X_train, y_train, cv=cv)
        
        # Log the results to WandB:
        wandb.log({f'{model_name}_CV_Scores': cv_scores.tolist(), f'{model_name}_Mean_CV_Score': np.mean(cv_scores)})
        
        results.append({
            'model_name': model_name,
            'cv_scores': cv_scores,
            'mean_cv_score': np.mean(cv_scores),
            #'parameters': {
                #'scale_pos_weight': class_weight if model_name == 'XGBClassifier' else None
            #}
        })

    return results

xgb = XGBClassifier(random_state =1)

models = [
    xgb,
    GaussianNB(),
    LogisticRegression(max_iter=2000),
    RandomForestClassifier(random_state=1),
    GradientBoostingClassifier(random_state=1),
    KNeighborsClassifier(),
    DecisionTreeClassifier(random_state=1),
    SVC(probability=True)
]

results = run_cross_validation(models, X_subset, y_subset, cv=5)
print(results)'''

In [ ]:
'''results = [
    {'model_name': 'XGBClassifier', 'cv_scores': [0.61425, 0.6105, 0.61358333, 0.61016667, 0.61483333], 'mean_cv_score': 0.61267},
    {'model_name': 'GaussianNB', 'cv_scores': [0.63325, 0.62608333, 0.63175, 0.63225, 0.63616667], 'mean_cv_score': 0.6319},
    {'model_name': 'LogisticRegression', 'cv_scores': [0.63308333, 0.62625, 0.62925, 0.63233333, 0.635], 'mean_cv_score': 0.63118},
    {'model_name': 'RandomForestClassifier', 'cv_scores': [0.61175, 0.61183333, 0.613, 0.60966667, 0.61608333], 'mean_cv_score': 0.61247},
    {'model_name': 'GradientBoostingClassifier', 'cv_scores': [0.62625, 0.62375, 0.62791667, 0.62783333, 0.62983333], 'mean_cv_score': 0.62712},
    {'model_name': 'KNeighborsClassifier', 'cv_scores': [0.57091667, 0.56625, 0.566, 0.56675, 0.56666667], 'mean_cv_score': 0.56732},
    {'model_name': 'DecisionTreeClassifier', 'cv_scores': [0.53425, 0.54141667, 0.53958333, 0.53583333, 0.53825], 'mean_cv_score': 0.53787},
    {'model_name': 'SVC', 'cv_scores': [0.6285, 0.624, 0.62466667, 0.63075, 0.63091667], 'mean_cv_score': 0.62777}
]

df = pd.DataFrame(results)
print("We got this")
print(df)'''

In [ ]:
from sklearn.ensemble import VotingClassifier
xgb = XGBClassifier(random_state =1)
lr = LogisticRegression(max_iter=2000)
gnb = GaussianNB()
rf = RandomForestClassifier(random_state=1)
svc = SVC(probability=True)

voting_clf = VotingClassifier(estimators = [('lr',lr),
                                            ('gnb', gnb),
                                            ('rf', rf),
                                            ('svc',svc),
                                            ('xgb',xgb)], voting = 'soft')
print("voting classifier:")
#cv = cross_val_score(voting_clf,X_subset,y_subset,cv=5)
#print(cv)
#print(cv.mean())

In [ ]:
print("X_subset")
print(X_subset)
print("X_test")
print(X_test)
#print("X_sig")
#print(X_sig)
print("X_train_sig")
print(X_train_sig)
print("X_train_sig")

In [ ]:
voting_clf.fit(X_backup,y_backup)

In [ ]:
y_hat_base_vc = voting_clf.predict(X_test_scaled).astype(float)
submission = {'id': test.id , 'target': y_hat_base_vc}
submission = pd.DataFrame(data=submission)
submission.to_csv('submission.csv', index=False)